In [3]:
using LinearAlgebra
using Plots
import StatsBase
include("src/Trajectories.jl")
using .QuTaM
import Distributions, HypothesisTests

# Evaluation at Given times
To make sure 

In [2]:
################### Qubit decay example ######################
#### 1. Create the system instance
gamma = 1.5
n = 1
gamma1 = (n+1)*gamma 
gamma2 = (n)*gamma
sigma_m = [[0.0+0im, 0]  [1, 0]]
sigma_p = [[0.0+0im, 1]  [0, 0]]
sys = System( zeros(ComplexF64, 2, 2), # Hamiltonian
    [sqrt(gamma1)*sigma_m, sqrt(gamma2)*sigma_p ]) #Jump Operators
#### 2. Create the simulation parameters instance
psi0 = zeros(ComplexF64, 2)
psi0[2] = 1 # Initial condition
simulparams = SimulParameters(psi0,
    5.0, # Final time. Set very long so that all trajectories jump
    10, # seed
    1000, # Number of trajectories
    10_000, # Number of samples in the finegrid
    10, # Multiplier to use in the fine grid
    1e-3 # Tolerance for passing Dark state test
) 

SimulParameters(psi0=[0.0, 1.0]
nsamples=10000
seed=10
ntraj=1000)
multiplier=10.0
tf=5.0
dt=0.005
eps=0.001)

# Qubit Radiative Damping Test
To test this thing we will look at the following master equation:
$$ \dot{\rho} = -i\frac{\Delta\omega_a}{2}\left[\sigma_z, \rho \right] + \gamma \left( \sigma_-\rho\sigma_+ -\frac{1}{2}
\left\{\rho, \sigma_+\sigma_-\right\}\right) .$$ 
We have:

$$ L = \sqrt{\gamma}\sigma_-$$

$$ H = \frac{\Delta\omega_a}{2}\sigma_z$$

$$ H_e =\frac{\Delta\omega_a}{2}\sigma_z -  \gamma\frac{i}{2}|e\rangle \langle e | = \frac{1}{2}(\Delta\omega_a - i\gamma)|e\rangle \langle e | - \frac{\Delta\Omega_a}{2}|g\rangle\langle g|$$

$$ W(t \hspace{0.05cm}| \hspace{0.05cm} |e\rangle\langle e |) = \gamma e^{-\gamma t}$$

In [3]:
################### Data Generation
#EXAMPLE : RADIATIVE DAMPING ######################
#### 1. Create the system instance
gamma = 3
sigma_m = [[0.0+0im, 0] [1, 0]]
sys = System(zeros(ComplexF64, 2, 2), # Hamiltonian
    [sigma_m]) #Jump Operators
#### 2. Create the simulation parameters instance
psi0 = zeros(ComplexF64, 2)
psi0[2] = 1 # Initial condition
simulparams = SimulParameters(psi0,
    5.0, # Final time. Set very long so that all trajectories jump
    1, # seed
    10, # Number of trajectories
    10_000, # Number of samples in the finegrid
    3, # Multiplier to use in the fine grid
    1e-3 # Tolerance for passing Dark state test
) 

SimulParameters(psi0=[0.0, 1.0]
nsamples=10000
seed=1
ntraj=10)
multiplier=3.0
tf=5.0
dt=0.0015
eps=0.001)

In [2]:
QuTaM.states_at_jumps

LoadError: UndefVarError: `states_at_jumps` not defined in `Main.QuTaM`
Suggestion: check for spelling errors or missing imports.